## 환경 설정 및 초기화
* tokenizer (한글 형태소 분석기) 설치
* 한글 글꼴 설치
* 설치 후 런타임 재시작 필요

* 이후 파일 로드; mac에서 저장한 xlsx 파일은 읽지 못함. xls파일로 저장하여 로드할 것

In [3]:
# tokenizer
!pip install konlpy

# hangul font
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [1]:
!pwd
#!ls -al
!ls -al /content/drive/MyDrive/Colab\ Notebooks/nlp/kyobonlp
#MyDrive/Colab\ Notebooks/nlp/kyobonlp/trainingdata.xlsx

/content
total 133103
drwx------ 2 root root     4096 Dec 12 01:36 .git
-rw------- 1 root root 50697568 Dec 21  2016 ko.bin
-rw------- 1 root root 85362829 Dec 21  2016 ko.tsv
-rw------- 1 root root       38 Dec 12 01:36 README.md
-rw------- 1 root root    40214 Dec 12 03:47 trainingdata.csv
-rw------- 1 root root    91648 Dec 12 07:41 trainingdata.xls
-rw------- 1 root root    33871 Dec 12 07:38 trainingdata.xlsx
-rw------- 1 root root    64954 Dec 13 10:04 word2vec.ipynb


In [18]:
# data import
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/nlp/kyobonlp/trainingdata.xls")
df

,id,label,text
0,1,B,올해에는 많은 부서들이 예산보다 초과 지출했다.
1,2,B,그 사업을 위한 예산으로 1억원이 책정되어 있다
2,3,B,우리는 예산이 아주 빠듯하다
3,4,B,그 부서의 예산은 손댈 수 없다
4,5,B,우리가 올해는 예산보다 돈을 덜 썼다.
...,...,...,...
448,449,C,이 제품과 비교제품의 차이점을 자세히 알고 싶습니다.
449,450,C,이 제품은 비교제품에 비해 어떤 점이 더 다른가요?
450,451,C,제가 검토한 비교 제품과 이 제품을 비교해보면 어떤 차이가 있나요?
451,452,C,"비교제품 대비, 제가 이 제품을 구매해야 하는 이유가 무엇인가요?"


In [19]:
# preprocess; exclude special symbols
df['text'] = df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
df['text'] = df['text'].str.replace(" +"," ")
df

<ipython-input-19-08a603e07a5d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
<ipython-input-19-08a603e07a5d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(" +"," ")


,id,label,text
0,1,B,올해에는 많은 부서들이 예산보다 초과 지출했다
1,2,B,그 사업을 위한 예산으로 억원이 책정되어 있다
2,3,B,우리는 예산이 아주 빠듯하다
3,4,B,그 부서의 예산은 손댈 수 없다
4,5,B,우리가 올해는 예산보다 돈을 덜 썼다
...,...,...,...
448,449,C,이 제품과 비교제품의 차이점을 자세히 알고 싶습니다
449,450,C,이 제품은 비교제품에 비해 어떤 점이 더 다른가요
450,451,C,제가 검토한 비교 제품과 이 제품을 비교해보면 어떤 차이가 있나요
451,452,C,비교제품 대비 제가 이 제품을 구매해야 하는 이유가 무엇인가요


## tokenization

In [20]:
# tokenize
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

#print("sklearn version is", sklearn.__version__)
tokenizer = Okt()

#전체 데이터에 대한 tf-idf를 구하려면 아래 코드 실행
true_corpus = " ".join(df['text'].values)

#tfidf_vector = TfidfVectorizer(tokenizer=tokenizer.nouns, min_df=2).fit([true_corpus, false_corpus])
#tfidf_result = tfidf_vector.transform([true_corpus, false_corpus])
tfidf_vector = TfidfVectorizer(tokenizer=tokenizer.nouns, min_df=1).fit([true_corpus])
tfidf_result = tfidf_vector.transform([true_corpus])
#vocab = tfidf_vector.vocabulary_
vocab = tfidf_vector.get_feature_names_out()
#print("vocab for debugging \n", vocab)

len(vocab)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


874

## tfidf for double check and decoration

In [5]:
# tfidf
tfidf_words = tfidf_result.toarray().argsort()
#print(tfidf_words[0])  # tokens

#tfidf_words[0] : true_corpus의 tf-idf 결과
#tfidf_words[1] : false_corpus의 tf-idf 결과
true_words = tfidf_words[0][::-1]  # reverse order
#print (true_words)

for word_index in true_words[:10]:
    print(vocab[word_index])

그
수
이
제품
것
결정
프로젝트
우리
회사
다른


In [6]:
real_true_words = []

for word_index in true_words:
  word = vocab[word_index]
  if len(word) >= 2:
    real_true_words.append(word)

print(len(real_true_words))
#real_true_words = real_true_words[:500]
print(real_true_words[:20])

750
['제품', '결정', '프로젝트', '우리', '회사', '다른', '계약', '대한', '비교', '제안', '저희', '예산', '비용', '만료', '권한', '관심', '가격', '기간', '위해', '이번']


## word2vec model

In [21]:
# tokenizing nouns only
df['tokens'] = df['text'].apply(tokenizer.nouns)
df#['tokens']

,id,label,text,tokens
0,1,B,올해에는 많은 부서들이 예산보다 초과 지출했다,"[올해, 부서, 예산, 초과, 지출]"
1,2,B,그 사업을 위한 예산으로 억원이 책정되어 있다,"[그, 사업, 위, 예산, 억원, 책정]"
2,3,B,우리는 예산이 아주 빠듯하다,"[우리, 예산, 아주, 빠듯하]"
3,4,B,그 부서의 예산은 손댈 수 없다,"[그, 부서, 예산, 수]"
4,5,B,우리가 올해는 예산보다 돈을 덜 썼다,"[우리, 올해, 예산, 돈, 덜]"
...,...,...,...,...
448,449,C,이 제품과 비교제품의 차이점을 자세히 알고 싶습니다,"[이, 제품, 비교, 제품, 차이점, 알]"
449,450,C,이 제품은 비교제품에 비해 어떤 점이 더 다른가요,"[이, 제품, 비교, 제품, 점, 더, 가요]"
450,451,C,제가 검토한 비교 제품과 이 제품을 비교해보면 어떤 차이가 있나요,"[제, 검토, 비교, 제품, 이, 제품, 비교, 차이]"
451,452,C,비교제품 대비 제가 이 제품을 구매해야 하는 이유가 무엇인가요,"[비교, 제품, 대비, 제, 이, 제품, 구매, 이유, 무엇]"


In [8]:
# skip-gram model
from gensim.models import Word2Vec
# 파라메터 설명(https://radimrehurek.com/gensim/models/word2vec.html)
#sentences : 학습에 사용될 문장들 혹은 토크나이징 된 문장들
#vector_size : 벡터의 차원(크기)
#window : 모델이 단어의 관계성을 따지는 window 크기
#min_count : 이 횟수 이하로 언급된 단어는 제외함
#sg : 1일 때 skip-gram, 그 외에는 CBOW
sg_model = Word2Vec(sentences = df['tokens'], vector_size = 100, window = 5, min_count = 3, sg = 1)
sg_model.wv.similar_by_word('예산')

[('저희', 0.6744961738586426),
 ('수', 0.6413315534591675),
 ('서비스', 0.614935040473938),
 ('제품', 0.6100627779960632),
 ('제', 0.6054443120956421),
 ('이', 0.592955470085144),
 ('를', 0.5895404815673828),
 ('것', 0.5780844688415527),
 ('바로', 0.5769188404083252),
 ('고객', 0.5684555172920227)]

In [ ]:
sg_model.wv.get_vector('예산')

In [9]:
# CBOW model
cbow_model = Word2Vec(sentences = df['tokens'], vector_size = 100, window = 5, min_count = 5, sg = 0)
cbow_model.wv.similar_by_word('예산')

[('제안', 0.301333487033844),
 ('이해', 0.2750489413738251),
 ('납품', 0.26256394386291504),
 ('모든', 0.25210604071617126),
 ('거', 0.24639342725276947),
 ('일이', 0.24362513422966003),
 ('무엇', 0.23965321481227875),
 ('계획', 0.23377569019794464),
 ('날짜', 0.23047234117984772),
 ('진행', 0.22674009203910828)]

In [17]:
# pretraining model
print(gensim.__version__)
pre_model = Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/nlp/kyobonlp/ko.bin')
pre_model.wv.similar_by_word('예산')

4.3.2


ERROR:gensim.models.word2vec:Model load error. Was model saved using code from an older Gensim Version? Try loading older model using gensim-3.8.3, then re-saving, to restore compatibility with current code.


AttributeError: ignored